In [1]:
from testsavant.guard import InputGuard, OutputGuard
from testsavant.guard.input_scanners import Anonymize
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
def dummy_llm(input):
    return input
long_article = "The company offered Mr Musk a substantial equity stake in Tesla, which he accepted. This is how companies like Tesla attract top talent and compete with established automakers. " * 4
print(len(long_article))

ts_api = InputGuard(
    API_KEY=os.environ.get("TEST_SAVANT_API_KEY"),
    PROJECT_ID=os.environ.get("TEST_SAVANT_PROJECT_ID"),
    remote_addr=os.environ.get("TEST_SAVANT_REMOTE_ADDR")
)
scanner = Anonymize(tag="base", threshold=0.5, use_faker=False, redact=False)
ts_api.add_scanner(scanner)

708


In [3]:
prompt = "Mr Don Smith is a software engineer. He works at a company called TechCorp and he is a colleague of Jane Doe. They are working on a project together. The project is about developing a new software application that will help people manage their tasks more efficiently. Mr Smith is very excited about this project and believes it will be a great success at google"
result = ts_api.scan(prompt)
if result.is_valid:
    print("Prompt is valid")
else:
    print("Prompt is invalid")

Prompt is invalid


In [4]:
result.sanitized_prompt


'Mr [REDACTED_PERSON_1] is a software engineer. He works at a company called TechCorp and he is a colleague of [REDACTED_PERSON_2] Doe. They are working on a project together. The project is about developing a new software application that will help people manage their tasks more efficiently. Mr [REDACTED_PERSON_3] is very excited about this project and believes it will be a great success at google'

In [5]:
len(prompt), len(result.sanitized_prompt)

(361, 400)

In [6]:
len(prompt.split()), len(result.sanitized_prompt.split())

(65, 64)

In [7]:
xprompt = prompt + " "  + long_article
result = ts_api.scan(xprompt)
len(xprompt.split()), len(result.sanitized_prompt.split())

(177, 176)

In [8]:
nprompt = "Mr Smith went to the Tesla factory to launch the new Tesla Model Y, early in the morning"
for i in range(1):
    result = ts_api.scan(nprompt)
    print(result.is_valid)
    print(result.sanitized_prompt)

False
Mr [REDACTED_PERSON_3] went to the Tesla factory to launch the new Tesla Model Y, early in the morning


In [9]:
# # Use a pipeline as a high-level helper
# from transformers import pipeline

# pipe = pipeline("ner", model="Isotonic/distilbert_finetuned_ai4privacy_v2")

In [10]:
# pipe(nprompt)